In [3]:
from box import ConfigBox

In [5]:
d2 = ConfigBox({"key": "value", "key2": "value2"})

In [5]:
import os
os.chdir("../")

In [10]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from datasets import load_dataset, load_from_disk, load_metric
from textSummarizer.utils.common import read_yaml
from textSummarizer.constants import *

import torch



config = read_yaml(CONFIG_FILE_PATH)
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(config.model_evaluation.tokenizer_path)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(config.model_evaluation.model_path).to(device)

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 48}


dataset_samsum_pt = load_from_disk(config.model_evaluation.data_path)
sample_text = dataset_samsum_pt["test"][0]["dialogue"]
reference = dataset_samsum_pt["test"][0]["summary"]

pipe = pipeline("summarization", model=model_pegasus, tokenizer=tokenizer, device=device)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

[2023-11-10 16:43:03: INFO: textSummarizerLogger: Read yaml file: config\config.yaml loaded successfully]
Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Larry called Betty last time they were at the park together. He's very nice. Hannah wants Amanda to text him. Amanda says she'd rather she text him.
